## Finding the most profitable towns in a CCAA given a max price

In [1]:
import pandas as pd
import math
from bs4 import BeautifulSoup
import requests
import nums_from_string as nfs
import numpy as np
from re import search
from random import randint
from time import sleep
import openpyxl
import xlrd
import lxml
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns

#### Importing geo data from csv file

In [2]:
geo_data = pd.read_csv('/Users/ignaciolorenzoqueralt/Documents/Ironhack/Final Project/data/province-town-n_props/geo-data_2021.10.23.csv')

In [3]:
geo_data.head(3)

,Unnamed: 0,town,n_properties,province,ccaa
0,0,alegria___dulantzi,26,alava,pais_vasco
1,1,amurrio,6,alava,pais_vasco
2,2,anana,2,alava,pais_vasco


#### Getting data about properties for sale to query accordingly properties for rent 

Importing properties for sale csv

In [4]:
data_sale_properties = pd.read_csv('/Users/ignaciolorenzoqueralt/Documents/Ironhack/Final Project/properties/sale/2021.11.08_cataluña_90000_20.csv', engine = 'python')
data_sale_properties = data_sale_properties.loc[:, ~data_sale_properties.columns.str.contains('^Unnamed')]

In [5]:
def checking_nulls(df):
    # This function shows which columns have null values and returns a df with only nulls
    for c in df.columns:
        null_count = df[c].isnull().sum()
        if null_count > 0:
            print ("The column ", c, " has ", null_count, " null values")
    nulls = df[df.isna().any(axis=1)]
    return nulls.head(3)

In [7]:
data_sale_properties = data_sale_properties.dropna()

In [6]:
checking_nulls(data_sale_properties)

The column  name  has  8  null values
The column  area  has  8  null values
The column  neighborhood  has  8  null values
The column  geo_town  has  25  null values
The column  m2  has  25  null values
The column  n_rooms  has  25  null values
The column  n_bath  has  25  null values
The column  price_m2  has  25  null values
The column  price  has  25  null values
The column  price_reduction  has  25  null values
The column  opportunity  has  25  null values
The column  last_update  has  25  null values
The column  description  has  25  null values
The column  url  has  42  null values
The column  province  has  42  null values
The column  ccaa  has  42  null values


,name,area,neighborhood,geo_town,m2,n_rooms,n_bath,price_m2,price,price_reduction,opportunity,last_update,description,url,province,ccaa
600,Piso C/ jaume i. Loft en venta en castellar d...,Castellar del Vallès,Poble,castellar_del_valles,90.0,3.0,1.0,833.0,75000.0,14000.0,no,3.0,Amida Immobles comercializa esta gran oportuni...,None,None,None
601,https://www.habitaclia.com/comprar-piso-loft_e...,barcelona,cataluña,None,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,None,None
759,Piso en Congost. Piso en venta en can bassa-...,Granollers,Congost,granollers,72.0,2.0,1.0,1.0,89550.0,0.0,no,2.0,Piso a reformar en Granollers!!,None,None,None


In [8]:
data_sale_properties.head(3)

,name,area,neighborhood,geo_town,m2,n_rooms,n_bath,price_m2,price,price_reduction,opportunity,last_update,description,url,province,ccaa
0,"Piso en Plaça sa boada, 35. Oportunidad en e...",Arenys de Mar,Centre,arenys_de_mar,50.0,3.0,1.0,1.0,94900.0,100.0,yes,0.0,Tenemos en exclusiva esta vivienda visitas con...,https://www.habitaclia.com/comprar-piso-oportu...,barcelona,cataluña
1,Apartamento Carrer avall (d´). Apartamento en...,Arenys de Mar,Centre,arenys_de_mar,41.0,1.0,1.0,2.0,89000.0,0.0,no,4.0,Disseny i habitatge les presenta este fantásti...,https://www.habitaclia.com/comprar-apartamento...,barcelona,cataluña
2,Piso Carrer camí de la pietat. Piso economico...,Arenys de Mar,Centre,arenys_de_mar,50.0,3.0,1.0,1.0,88700.0,0.0,no,0.0,"Piso de 50 m2, con comedor de 15 m2, cocina in...",https://www.habitaclia.com/comprar-piso-econom...,barcelona,cataluña


#### Cleaning imported data

In [9]:
def convert_to_num(df, column):
    for i,value in enumerate(df[column]):
        df[column] = df[column].astype(int)

In [10]:
convert_to_num(data_sale_properties, column = 'm2')
convert_to_num(data_sale_properties, column = 'n_rooms')
convert_to_num(data_sale_properties, column = 'n_bath')

#### Exploring numerical features

In [11]:
features = data_sale_properties.filter(['m2','n_bath', 'n_rooms'], axis=1)

In [12]:
data_sale_properties.describe().T

,count,mean,std,min,25%,50%,75%,max
m2,7102.0,71.307519,31.499712,1.0,54.0,67.0,81.0,900.0
n_rooms,7102.0,2.602084,11.859891,1.0,2.0,3.0,3.0,999.0
n_bath,7102.0,1.290200,11.846629,1.0,1.0,1.0,1.0,999.0
price_m2,7102.0,244.609547,362.379118,1.0,1.0,1.0,600.0,999.0
price,7102.0,78879.009012,19058.562460,800.0,68000.0,84000.0,94375.0,108000.0
price_reduction,7102.0,2771.249085,24450.224001,0.0,0.0,0.0,3000.0,954000.0
last_update,7102.0,4.053647,5.224175,0.0,0.0,3.0,6.0,59.0


#### Getting the maximum price for the rent of the properties that we want to scrape

We get the 50% quantile price of the imported properties listed for sale and we extrapolate an expected rent price for those properties assuming a 12% rent. Since we want a maximum price, we use a generous profitability ratio (12%).

In [13]:
max_prices = [300, 400, 500, 600, 700, 800, 900, 1000, 1100, 1200, 1500, 1800, 2000, 3000, 4000, 5000]

In [14]:
max_price = data_sale_properties.price.quantile(0.5)*0.12/12

if max_price < 1200:
    max_price = max_price/100
    max_price = math.ceil(max_price)*100
elif max_price > 1200 & max_price < 1500:
    max_price = 1500
elif max_price > 1500 & max_price < 1800:
    max_price = 1800
elif max_price > 1800 & max_price < 2000:
    max_price = 2000
elif max_price > 2000 & max_price < 3000:
    max_price = 3000
elif max_price > 3000 & max_price < 4000:
    max_price = 4000
elif max_price > 4000 & max_price < 5000:
    max_price = 5000
elif max_price > 5000:
    max_price = 5000

In [15]:
max_price

900

#### Get input from user

In [16]:
ccaa_lst = geo_data['ccaa'].unique().tolist()
ccaa = input("ccaa: ")
while ccaa not in ccaa_lst:
    ccaa = input("There was no match between your input and our ccaa, try again: ")

ccaa:  cataluña


In [17]:
num_properties_per_town = ""
while num_properties_per_town == "":
    try:
        
        num_properties_per_town = int(input("Minimum number of properties per town: "))
    except: 
        num_properties_per_town = ""

Minimum number of properties per town:  1


In [19]:
filtered_df = geo_data[(geo_data['ccaa'] == ccaa) & (geo_data['n_properties'] > num_properties_per_town)]

#### List of towns that we need to extract data from

In [20]:
towns = filtered_df[filtered_df['ccaa'] == ccaa].town.to_list()

Hay que hacer un for loop que para cada town saque el df y le haga un append a el global.

#### Extracting data from each town

In [22]:
property = "alquiler"
municipio = "hospitalet_de_llobregat"
ascensor = "-ascensor"

habitaciones = "hab="+"1"
baños = "&"+"ban="+"1"
maximum_price = "pmax="+str(max_price)
metros = "m2="+"50"

In [23]:
url = "https://www.habitaclia.com/"+property+ascensor+"-"+municipio+".htm?"+habitaciones+baños+maximum_price

In [24]:
# Creation of the df to which we will append the properties of the selected towns.
name = []
town = []
area = []
neighborhood = []
geo_town = []
features = []
m2 = []
n_rooms = []
n_bath = []
price_m2 = []
description = []
price = []
opportunity = []
price_reduction = []
opportunity = []
last_update = []
url = []

x = min(len(name), len(town), len(area), len(neighborhood), len(geo_town), len(description), len(price), len(last_update), len(url))
dct = {'name': name[:x], 'town': town[:x], 'area': area[:x], 'neighborhood': neighborhood[:x], 'geo_town':geo_town[:x],'m2': m2[:x], 'n_rooms': n_rooms[:x], 'n_bath': n_bath[:x], 'price_m2': price_m2[:x] ,'price': price[:x], 'price_reduction': price_reduction[:x], 'opportunity':opportunity[:x], 'last_update': last_update[:x],  'description': description[:x], 'url':url[:x] }
df = pd.DataFrame.from_dict(dct)

# Pulling properties from each town in the previously defined list:

for t in towns:
    
    # Getting the number of properties for that town to see how many pages do we need to scrape.
    url = "https://www.habitaclia.com/"+property+"-"+t+".htm?"+maximum_price
    r = requests.get(url)
    r.status_code
    soup = BeautifulSoup(r.content, 'html.parser')
    
    try:
        total_results = int(soup.find('h2', attrs={'class': 'f-right'}).find('span').get_text().replace(".",""))
        pages = range(int(math.floor(total_results/16))+1)
        properties = []

        # Adding all the properties listed in each page to the list.
        for p in pages:
            sleep(randint(2,5))
            url = "https://www.habitaclia.com/"+property+"-"+t+"-"+str(p)+".htm?"+maximum_price
            #print(url)
            try:
                r = requests.get(url)
                soup = BeautifulSoup(r.content, 'html.parser')
                properties += soup.find_all('div', attrs={'class': 'list-item-info'})
                del properties[-1] # Last item is an ad
            except:
                print('Error on page', p)
            #print('town: ', t, " page: ", p)

        # Creating a list for each piece of information I want to extract from each property.
        name = []
        town = []
        area = []
        neighborhood = []
        geo_town = []
        features = []
        m2 = []
        n_rooms = []
        n_bath = []
        price_m2 = []
        description = []
        price = []
        opportunity = []
        price_reduction = []
        opportunity = []
        last_update = []
        url = []

        # Getting the information from each property.
        for i,properties in enumerate(properties):
            # Each feature is set as empty prior to being defined. This way we avoid errors when a feature is not available for a certain property.
            name_temp = ""
            town_temp = ""
            area_temp = ""
            neighborhood_temp = ""
            geo_town_temp = ""
            m2_temp = ""
            n_rooms_temp = ""
            n_bath_temp = ""
            price_m2_temp = ""
            price_temp = ""
            opportunity_temp = ""
            price_reduction_temp = ""
            description_temp = ""
            last_update_temp = ""
            url_temp = ""

            # other_location enables us to differ between listed properties vs suggested properties, which appear when there are very few properties for one town. We want to avoid them as they are nearby properties not belonging to our target town.
            other_location = properties.find('span', attrs={'class': 'ady-relationship'})
            if other_location is None:
                other_locations_properties = ""
            else: 
                #print(i)
                other_locations_properties = other_location.get_text(strip=True).find('Se encuentra en')

            # Now I am skipping all the properties that are suggested so as to not append them to the df.
            if other_locations_properties == 0:
                pass
            else:
                try:
                    # Extracting the features of a property and saving them in a temporary variable.
                    name_temp = properties.find('h3', attrs={'class': 'list-item-title'}).get_text(strip=True)
                    town_temp = properties.find('p', attrs={'class': 'list-item-location'}).get_text(strip=True).split("-",1)[0].strip().replace("Ver mapa","")
                    area_temp = properties.find('p', attrs={'class': 'list-item-location'}).get_text(strip=True).replace('/','-').strip().replace("Ver mapa","").split("-",1)[0]
                    neighborhood_temp = properties.find('p', attrs={'class': 'list-item-location'}).get_text(strip=True).replace('/','-').strip().replace("Ver mapa","").split("-",1)[1].strip()
                    geo_town_temp = t
                    m2_temp = nfs.get_nums(properties.find('p', attrs={'class': 'list-item-feature'}).get_text(strip=True).split("-")[0])[0] 
                    n_rooms_temp = nfs.get_nums(properties.find('p', attrs={'class': 'list-item-feature'}).get_text(strip=True).split("-")[1])[0]
                    n_bath_temp = nfs.get_nums(properties.find('p', attrs={'class': 'list-item-feature'}).get_text(strip=True).split("-")[2])[0] 
                    price_m2_temp = nfs.get_nums(properties.find('p', attrs={'class': 'list-item-feature'}).get_text(strip=True).split("-")[3])[0] 
                    raw_price_temp = properties.find('article', attrs={'class': 'list-item-price'}).get_text()
                    if search("Oportunidad", raw_price_temp):
                        if search("ha bajado", raw_price_temp): 
                            price_temp = nfs.get_nums((properties.find('article', attrs={'class': 'list-item-price'}).get_text()).replace(".",""))[0]
                            price_reduction_temp = nfs.get_nums((properties.find('article', attrs={'class': 'list-item-price'}).get_text()).replace(".",""))[1]
                            opportunity_temp = "yes"
                        else:
                            price_temp = nfs.get_nums((properties.find('article', attrs={'class': 'list-item-price'}).get_text()).replace(".",""))[0]
                            price_reduction_temp = "0"
                            opportunity_temp = "yes"
                    elif search("ha bajado", raw_price_temp):
                        price_temp = nfs.get_nums((properties.find('article', attrs={'class': 'list-item-price'}).get_text()).replace(".",""))[0]
                        price_reduction_temp = nfs.get_nums((properties.find('article', attrs={'class': 'list-item-price'}).get_text()).replace(".",""))[1]      
                        opportunity_temp = "no"
                    else: 
                        price_temp = nfs.get_nums((properties.find('article', attrs={'class': 'list-item-price'}).get_text()).replace(".",""))[0]
                        price_reduction_temp = "0"
                        opportunity_temp = "no"
                    description_temp = properties.find('p', attrs={'class': 'list-item-description'}).get_text(strip=True)
                    last_update_temp = nfs.get_nums(properties.find('span', attrs={'class': 'list-item-date'}).get_text(strip=True))
                    url_temp = properties.find('h3', attrs={'class': 'list-item-title'}).find('a').get('href')

                    # Appending temporary variables features to their corresponding list.
                    name.append(name_temp)
                    town.append(town_temp)
                    area.append(area_temp)
                    neighborhood.append(neighborhood_temp)
                    geo_town.append(geo_town_temp)
                    m2.append(m2_temp)
                    n_rooms.append(n_rooms_temp)
                    n_bath.append(n_bath_temp)
                    price_m2.append(price_m2_temp)
                    price.append(price_temp)
                    opportunity.append(opportunity_temp)
                    price_reduction.append(price_reduction_temp)
                    description.append(description_temp)
                    last_update.append(last_update_temp)
                    url.append(url_temp)

                except:
                    # In case we may encounter an error, we print the features of each property to find the bug.
                    '''
                    print('------------------------------------')
                    print('nombre: ', name_temp)
                    print('town_temp: ', town_temp)
                    print('area_temp: ', area_temp)
                    print('m2_temp: ', m2_temp)
                    print('n_rooms_temp: ', n_rooms_temp)
                    print('n_bath_temp: ', n_bath_temp)
                    print('price_m2_temp: ', price_m2_temp)
                    print('price_temp: ', price_temp)
                    print('opportunity_temp: ', opportunity_temp)
                    print('price_reduction_temp: ', price_reduction_temp)
                    print('description_temp: ', description_temp)
                    print('last_update_temp: ', last_update_temp)
                    print('url_temp: ', url_temp)
                    print('------------------------------------')
                    '''

        x_town = min(len(name), len(town), len(area), len(neighborhood), len(geo_town), len(description), len(price), len(last_update), len(url))
        dct_town = {'name': name[:x_town], 'town': town[:x_town], 'area': area[:x_town], 'neighborhood': neighborhood[:x_town], 'geo_town':geo_town[:x_town],'m2': m2[:x_town], 'n_rooms': n_rooms[:x_town], 'n_bath': n_bath[:x_town], 'price_m2': price_m2[:x_town] ,'price': price[:x_town], 'price_reduction': price_reduction[:x_town], 'opportunity':opportunity[:x_town], 'last_update': last_update[:x_town],  'description': description[:x_town], 'url':url[:x_town] }
        df_town = pd.DataFrame.from_dict(dct_town)
        df = df.append(df_town, ignore_index = True)

    except:
        print("no properties found at: ", url)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223


#### Output from the web scrapping

In [25]:
df.shape

(126, 15)

In [26]:
df1 = df.copy()

#### Cleaning the df

In [27]:
df1 = df1.merge(geo_data, left_on='geo_town', right_on='town', how='left')
df1 = df1.drop(["town_x", "town_y", "n_properties"], axis=1)
df1 = df1.loc[:, ~df1.columns.str.contains('^Unnamed')]

In [28]:
def clean_last_update():
    for i,n in enumerate(df1['last_update']):
        try:
            df1['last_update'][i] = df1['last_update'][i][0]
        except:
            df1['last_update'][i] = 'null'
clean_last_update()

<ipython-input-28-b04376e5d5db>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['last_update'][i] = 'null'
<ipython-input-28-b04376e5d5db>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['last_update'][i] = df1['last_update'][i][0]


In [29]:
numericals = ['m2', 'n_rooms', 'n_bath', 'price_m2', 'price', 'price_reduction', 'last_update']
def anytype_to_numerical(df, columns = []):
    for c in columns:
        if df[c].dtypes == 'float64':
            df[c] = df[c].astype(int)
anytype_to_numerical(df1, columns = numericals)

In [47]:
df1.head(3)

,name,area,neighborhood,geo_town,m2,n_rooms,n_bath,price_m2,price,price_reduction,opportunity,last_update,description,url,province,ccaa
0,Alquiler Piso en Creu Alta. Piso semi nuevo ...,Sabadell,Creu Alta,sabadell,47,1,1,15,725,0,no,4,"Piso en la creu alta de 47 m2 según catastro, ...",https://www.habitaclia.com/alquiler-piso-semi_...,barcelona,cataluña
1,Alquiler Piso en Centre. Pis en lloguer al c...,Sabadell,Centre,sabadell,90,2,2,9,877,0,no,null,"GESTIONS IMMOBILIÀRIES NAYACH comercialitza, P...",https://www.habitaclia.com/alquiler-piso-pis_e...,barcelona,cataluña
2,"Alquiler Apartamento en Centro ciudad, s/n. ...",Sabadell,Centre,sabadell,70,2,1,10,750,0,yes,5,Bonito apartamento compuesto de dos habitacion...,https://www.habitaclia.com/alquiler-apartament...,barcelona,cataluña


#### Getting info about lifts

In [35]:
# Creation of the df to which we will append the properties of the selected towns.
name = []
town = []
area = []
neighborhood = []
geo_town = []
features = []
m2 = []
n_rooms = []
n_bath = []
price_m2 = []
description = []
price = []
opportunity = []
price_reduction = []
opportunity = []
last_update = []
url = []
lift = []

x = min(len(name), len(town), len(area), len(neighborhood), len(geo_town), len(description), len(price), len(last_update), len(url), len(lift))
dct = {'name': name[:x], 'town': town[:x], 'area': area[:x], 'neighborhood': neighborhood[:x], 'geo_town':geo_town[:x],'m2': m2[:x], 'n_rooms': n_rooms[:x], 'n_bath': n_bath[:x], 'price_m2': price_m2[:x] ,'price': price[:x], 'price_reduction': price_reduction[:x], 'opportunity':opportunity[:x], 'last_update': last_update[:x],  'description': description[:x], 'url':url[:x], 'lift':lift[:x]}
df = pd.DataFrame.from_dict(dct)

# Pulling properties from each town in the previously defined list:

for t in towns:
    
    # Getting the number of properties for that town to see how many pages do we need to scrape.
    url = "https://www.habitaclia.com/"+property+"-"+"viviendas-ascensor"+"-"+t+".htm?"+maximum_price
    r = requests.get(url)
    r.status_code
    soup = BeautifulSoup(r.content, 'html.parser')
    try:
        total_results = int(soup.find('h2', attrs={'class': 'f-right'}).find('span').get_text().replace(".",""))
        pages = range(int(math.floor(total_results/16))+1)
        properties = []

        # Adding all the properties listed in each page to the list.
        for p in pages:
            sleep(randint(2,5))
            url = "https://www.habitaclia.com/"+property+"-"+"viviendas-ascensor"+"-"+t+"-"+str(p)+".htm?"+maximum_price
            try:
                r = requests.get(url)
                soup = BeautifulSoup(r.content, 'html.parser')
                properties += soup.find_all('div', attrs={'class': 'list-item-info'})
                del properties[-1] # Last item is an ad
            except:
                print('Error on page', p)
                print('town: ', t, " page: ", p, "url: ", url)

        # Creating a list for each piece of information I want to extract from each property.
        name = []
        town = []
        area = []
        neighborhood = []
        geo_town = []
        features = []
        m2 = []
        n_rooms = []
        n_bath = []
        price_m2 = []
        description = []
        price = []
        opportunity = []
        price_reduction = []
        opportunity = []
        last_update = []
        url = []

        # Getting the information from each property.
        for i,properties in enumerate(properties):
            #print(i)
            # Each feature is set as empty prior to being defined. This way we avoid errors when a feature is not available for a certain property.
            name_temp = ""
            town_temp = ""
            area_temp = ""
            neighborhood_temp = ""
            geo_town_temp = ""
            m2_temp = ""
            n_rooms_temp = ""
            n_bath_temp = ""
            price_m2_temp = ""
            price_temp = ""
            opportunity_temp = ""
            price_reduction_temp = ""
            description_temp = ""
            last_update_temp = ""
            url_temp = ""
            lift_temp = ""

            # other_location enables us to differ between listed properties vs suggested properties, which appear when there are very few properties for one town. We want to avoid them as they are nearby properties not belonging to our target town.
            other_location = properties.find('span', attrs={'class': 'ady-relationship'})
            if other_location is None:
                other_locations_properties = ""
            else: 
                #print(i)
                other_locations_properties = other_location.get_text(strip=True).find('Se encuentra en')

            # Now I am skipping all the properties that are suggested so as to not append them to the df.
            if other_locations_properties == 0:
                pass
            else:
                try:
                    # Extracting the features of a property and saving them in a temporary variable.
                    name_temp = properties.find('h3', attrs={'class': 'list-item-title'}).get_text(strip=True)
                    town_temp = properties.find('p', attrs={'class': 'list-item-location'}).get_text(strip=True).split("-",1)[0].strip().replace("Ver mapa","")
                    area_temp = properties.find('p', attrs={'class': 'list-item-location'}).get_text(strip=True).replace('/','-').strip().replace("Ver mapa","").split("-",1)[0]
                    neighborhood_temp = properties.find('p', attrs={'class': 'list-item-location'}).get_text(strip=True).replace('/','-').strip().replace("Ver mapa","").split("-",1)[1].strip()
                    geo_town_temp = t
                    m2_temp = nfs.get_nums(properties.find('p', attrs={'class': 'list-item-feature'}).get_text(strip=True).split("-")[0])[0] 
                    n_rooms_temp = nfs.get_nums(properties.find('p', attrs={'class': 'list-item-feature'}).get_text(strip=True).split("-")[1])[0]
                    n_bath_temp = nfs.get_nums(properties.find('p', attrs={'class': 'list-item-feature'}).get_text(strip=True).split("-")[2])[0] 
                    price_m2_temp = nfs.get_nums(properties.find('p', attrs={'class': 'list-item-feature'}).get_text(strip=True).split("-")[3])[0] 
                    raw_price_temp = properties.find('article', attrs={'class': 'list-item-price'}).get_text()
                    if search("Oportunidad", raw_price_temp):
                        if search("ha bajado", raw_price_temp): 
                            price_temp = nfs.get_nums((properties.find('article', attrs={'class': 'list-item-price'}).get_text()).replace(".",""))[0]
                            price_reduction_temp = nfs.get_nums((properties.find('article', attrs={'class': 'list-item-price'}).get_text()).replace(".",""))[1]
                            opportunity_temp = "yes"
                        else:
                            price_temp = nfs.get_nums((properties.find('article', attrs={'class': 'list-item-price'}).get_text()).replace(".",""))[0]
                            price_reduction_temp = "0"
                            opportunity_temp = "yes"
                    elif search("ha bajado", raw_price_temp):
                        price_temp = nfs.get_nums((properties.find('article', attrs={'class': 'list-item-price'}).get_text()).replace(".",""))[0]
                        price_reduction_temp = nfs.get_nums((properties.find('article', attrs={'class': 'list-item-price'}).get_text()).replace(".",""))[1]      
                        opportunity_temp = "no"
                    else: 
                        price_temp = nfs.get_nums((properties.find('article', attrs={'class': 'list-item-price'}).get_text()).replace(".",""))[0]
                        price_reduction_temp = "0"
                        opportunity_temp = "no"
                    description_temp = properties.find('p', attrs={'class': 'list-item-description'}).get_text(strip=True)
                    last_update_temp = nfs.get_nums(properties.find('span', attrs={'class': 'list-item-date'}).get_text(strip=True))
                    url_temp = properties.find('h3', attrs={'class': 'list-item-title'}).find('a').get('href')
                    lift_temp = 'yes'

                    # Appending temporary variables features to their corresponding list.
                    name.append(name_temp)
                    town.append(town_temp)
                    area.append(area_temp)
                    neighborhood.append(neighborhood_temp)
                    geo_town.append(geo_town_temp)
                    m2.append(m2_temp)
                    n_rooms.append(n_rooms_temp)
                    n_bath.append(n_bath_temp)
                    price_m2.append(price_m2_temp)
                    price.append(price_temp)
                    opportunity.append(opportunity_temp)
                    price_reduction.append(price_reduction_temp)
                    description.append(description_temp)
                    last_update.append(last_update_temp)
                    url.append(url_temp)
                    lift.append(lift_temp)

                except:
                    # In case we may encounter an error, we print the features of each property to find the bug.
                    '''
                    print('------------------------------------')
                    print('nombre: ', name_temp)
                    print('town_temp: ', town_temp)
                    print('area_temp: ', area_temp)
                    print('m2_temp: ', m2_temp)
                    print('n_rooms_temp: ', n_rooms_temp)
                    print('n_bath_temp: ', n_bath_temp)
                    print('price_m2_temp: ', price_m2_temp)
                    print('price_temp: ', price_temp)
                    print('opportunity_temp: ', opportunity_temp)
                    print('price_reduction_temp: ', price_reduction_temp)
                    print('description_temp: ', description_temp)
                    print('last_update_temp: ', last_update_temp)
                    print('url_temp: ', url_temp)
                    print('------------------------------------')
                    '''
        
        x_town = min(len(name), len(town), len(area), len(neighborhood), len(geo_town), len(description), len(price), len(last_update), len(url), len(lift))
        dct_town = {'name': name[:x_town], 'town': town[:x_town], 'area': area[:x_town], 'neighborhood': neighborhood[:x_town], 'geo_town':geo_town[:x_town],'m2': m2[:x_town], 'n_rooms': n_rooms[:x_town], 'n_bath': n_bath[:x_town], 'price_m2': price_m2[:x_town] ,'price': price[:x_town], 'price_reduction': price_reduction[:x_town], 'opportunity':opportunity[:x_town], 'last_update': last_update[:x_town],  'description': description[:x_town], 'url':url[:x_town], 'lift':lift[:x_town]}
        df_town = pd.DataFrame.from_dict(dct_town)
        df_lift = df.append(df_town, ignore_index = True)
        
        #Cleaning the final df
        #df = df[~df['description'].str.contains('nuda|sin cedula|sin cédula')]
    except:
        print("no properties found at: ", url)

In [36]:
df1_lift = df_lift.copy()

In [37]:
df1_lift = df1_lift.merge(geo_data, left_on='geo_town', right_on='town', how='left')
df1_lift = df1_lift.drop(["town_x", "town_y", "n_properties"], axis=1)
df1_lift = df1_lift.loc[:, ~df1_lift.columns.str.contains('^Unnamed')]

In [38]:
numericals = ['m2', 'n_rooms', 'n_bath', 'price_m2', 'price', 'price_reduction', 'last_update']

def anytype_to_numerical(df, columns = []):
    for c in columns:
        if df[c].dtypes == 'float64':
            df[c] = df[c].astype(int)

def clean_last_update():
    for i,n in enumerate(df1_lift['last_update']):
        try:
            df1_lift['last_update'][i] = df1_lift['last_update'][i][0]
        except:
            df1_lift['last_update'][i] = '0'

In [39]:
anytype_to_numerical(df1_lift, columns = numericals)
clean_last_update()

<ipython-input-38-a281638fdd74>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_lift['last_update'][i] = df1_lift['last_update'][i][0]
<ipython-input-38-a281638fdd74>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_lift['last_update'][i] = '0'


In [40]:
df1_lift= df1_lift[df1_lift['price']<int(max_price)]

In [41]:
df1_lift.shape

(70, 17)

In [43]:
list_names_with_lift = list(df1_lift['name'])

In [44]:
final_df = df1.combine_first(df1_lift)

In [45]:
final_df.shape

(126, 17)

#### Saving the scrapped df

In [ ]:
#today = datetime.now().strftime('%Y.%m.%d')
#df1.to_csv(path_or_buf = '/Users/ignaciolorenzoqueralt/Documents/Ironhack/Final Project/properties/rent/'+today+'_'+ccaa+'_'+str(max_price)+'_'+str(num_properties_per_town)+'.csv')